In [172]:
from PIL import Image
import os
import json
import random
import numpy as np


os.makedirs("./assets", exist_ok=True)
os.makedirs("./layers_data_json", exist_ok=True)
os.makedirs("./output", exist_ok=True)

path_project_dir="./assets/"

#Validation of 'schemas.json', check if some values make sense
def validate_schemas_file():
    with open("schemas.json") as user_file:
        config = json.load(user_file)

    check=True
    
    for k, weights_list in config["weight_traits"].items(): 
        weights=np.array(list(weights_list) ,dtype=np.float32)

        round_sum="{:.2f}".format(sum(weights))
        result= np.float32(round_sum)==np.float32(1.00)

        try:
            location=os.path.join("./assets/", k)
            files_list = os.listdir(location)
 
        except FileNotFoundError:
        
            print(f"Directory {location} doesn't exist.")
            return 0
        

        if not result:

            print(f'Total sum of weights for {k} is not 1!')
            check=False
        if not (len(files_list) == len(weights)):
            print(f'Directory or weights array of {k} with different sizes!')
            check=False

    for folder_corrente, nomi_sottocartelle, list_nomi_file in os.walk("assets"):
        if not nomi_sottocartelle:
            for f in list_nomi_file:
                img= Image.open(os.path.join(folder_corrente,f))
                w,h=img.size
                if not (w==config["width_images"] and h==config["height_images"]):
                    print(f'image {f} has sizes of {w}x{h}, sizes declared in config are {config["width_images"]}x{config["height_images"]}')
                    check=False
    if check:
        print("Images sizes checks: OK")
        print("Weights checks: OK ")
        print("No errors in the config schema")




#create a file contaning a path for each image's trait       
def create_jsons(path="assets"):
    data_json={}
    os.makedirs("layers_data_json",exist_ok=True)
    for folder_corrente, nomi_sottocartelle, list_nomi_file in os.walk(path):
        if not nomi_sottocartelle:
            for f in list_nomi_file:
        
                data_json[os.path.splitext(f)[0]]= os.path.normpath(os.path.join(folder_corrente, f))
        
            with open(os.path.join("layers_data_json",os.path.basename(folder_corrente) +".json"), "w") as json_file:
                json.dump(data_json, json_file, indent=2)
        
            data_json.clear()
    print("Files created in "+ "'/layers_data_json' " )

#create a random image combining random different traits for each layer based on their weights               
def new_image():
    
    with open("schemas.json") as user_file:
        data_layers = json.load(user_file)
    
    new_image={}
    
    select= np.random.choice(len(data_layers["weights_schema_in_collection"]), p= data_layers["weights_schema_in_collection"] )
    json_files=data_layers["layers_order"][select]

    for f in json_files:
        path= os.path.join("layers_data_json", f+".json")
        with open(path, 'r') as file_json:
            json_layer = json.load(file_json)

        i=np.random.choice(list(json_layer.keys()),p=data_layers["weight_traits"][os.path.splitext(f)[0]])
        new_image[os.path.splitext(os.path.basename(path))[0]]= json_layer[i]
        #new_image_data[os.path.splitext(os.path.basename(path))[0]]= i 

    return new_image
   

def generate_collection(num_to_generate:int ,name: str, symbol:str,description:str):
    
    with open("schemas.json") as file:
        config=json.load(file)
    width= config["width_images"]
    height= config["height_images"]

    os.makedirs("output",exist_ok=True)
    os.makedirs(os.path.join("output","img"),exist_ok=True)
    os.makedirs(os.path.join("output","metadata"),exist_ok=True)

    generated=[]
    while len(generated)<num_to_generate:
       image= new_image()
       if image not in generated:
           generated.append(image)
    
    
    data_img={}
    edition=0
    for image in generated:
        traits=[]
        base_image= Image.new("RGBA", (width,height), (255, 255, 255, 0))
        for k_layer, v_address in image.items():

            temp=Image.open(v_address).convert("RGBA") 
            base_image.paste(temp,(0,0),temp)

            second_lvl = os.path.basename(os.path.dirname(v_address))
            file_name = os.path.splitext(os.path.basename(v_address))[0]
            trait_obj={"trait_type": second_lvl , "value": file_name}
            traits.append(trait_obj)

        # metadata json
        data_img["name"]= f'{name} #{edition}'
        data_img["symbol"]= symbol
        data_img["description"]= description
        data_img["edition"]= edition
        data_img["image"]=f'{edition}.png'
        data_img["attributes"]=traits


        with open(os.path.join("output/metadata",f'{edition}.json'), "w") as json_file:
                json.dump(data_img, json_file, indent=2)
        
        
        base_image.save(os.path.join("output/img",f'{edition}.png'))
        edition+=1

    print("Process completed!")
    


    
def update_metadatas():
    
    metadatas=os.listdir("output/metadata")
    traits=[os.path.splitext(t)[0] for t in os.listdir("layers_data_json")]

    for m in metadatas:
        with open(os.path.join("output/metadata",m), "r+") as metaFile:
            oldFile= json.load(metaFile)
            
            values=[]
            to_add=[]

            for attribute in oldFile["attributes"]:
                
                values.append(attribute["trait_type"])
            
            for t in traits:
                if t not in values:
                    to_add.append(t)


            for t in to_add:

                temp={"trait_type": t, "value": "none"}

                oldFile["attributes"].append(temp)
            
            metaFile.seek(0)

            json.dump(oldFile,metaFile, indent=2)
            
    print("Metadatas updated!")
       

def create_counts()-> dict:

    attributes_count={}
    metadatas=os.listdir("output/metadata")
    
    for m in metadatas:
        with open(os.path.join("output/metadata",m), "r") as file:

            data=json.load(file)
            
            for attributes in data["attributes"]:
                attr_key= (attributes["trait_type"], attributes["value"])
                if attr_key not in attributes_count:
                   attributes_count[attr_key]=1
                else:
                    attributes_count[attr_key]+=1

    return attributes_count

def calculate_percentages(attributes_count:dict)-> dict:

    amount=len(os.listdir("output/metadata"))
    attributs_percentages={}
    for attribute in attributes_count:

        freq_percent= round((attributes_count[attribute]/amount)*100, 3)
        attributs_percentages[attribute]= freq_percent
    
    return attributs_percentages


def create_rich_metadata(attribute_count: dict, attribute_freq: dict) -> None:
    os.makedirs("output/rich_metadata", exist_ok=True )
    metadatas=os.listdir("output/metadata")
    
    for m in metadatas:
        with open(os.path.join("output/metadata",m), "r") as metaFile:
            data= json.load(metaFile)
            token_attributes = data['attributes']

            for attr in token_attributes:
                attr_key = (attr['trait_type'], attr['value'])

                count = attribute_count[attr_key]
                freq = attribute_freq[attr_key]

                attr['count'] = count
                attr['frequency'] = freq

        data['attributes'] = token_attributes

        with open(os.path.join("output/rich_metadata/", str(data["edition"])+".json"), 'w', encoding='utf-8') as outfile:
            json.dump(data, outfile, indent=2)   
    
    print("Rich metadatas created")


def calculate_mean()->dict:
    harmonic_means={}
    metadatas=os.listdir("output/rich_metadata")
    
    for m in metadatas:
        with open(os.path.join("output/rich_metadata",m), "r") as metaFile:
            data= json.load(metaFile)

            total_traits = len(data["attributes"])
            rarity_sum = 0

            for attr in data["attributes"]:
                rarity_sum += (1 / attr['count'])
            
            mean = total_traits / rarity_sum
            rounded_mean = round(mean, 3)

            harmonic_means[data["edition"]] = rounded_mean

        data['rarity'] = dict()
        data['rarity']['harmonic'] = dict()
        data['rarity']['harmonic']['score'] = rounded_mean

        # Opens the original json file and writes the new data
        with open(os.path.join("output/rich_metadata", m), 'w', encoding='utf-8') as outfile:
            json.dump(data, outfile, indent=2)

    return harmonic_means


def add_rarity_rank(harmonic_means: dict) -> None:
    sorted_tokens = sorted(harmonic_means.items(), key=lambda x: x[1])

    rank = 1
    for token in sorted_tokens:
        #print(token[0])

        with open(os.path.join("output/rich_metadata",str(token[0])+".json"), 'r+', encoding='utf-8') as infile:

            data = json.load(infile)
            data['rarity']['harmonic']['rank'] = rank

            infile.seek(0)
            json.dump(data, infile, indent=2)

        rank += 1
    
    print("Rarity ranks added, process completed!")


def summary_rank()->None:

    all_dicts=[]
    metadatas=os.listdir("output/rich_metadata")
    
    for  m in metadatas:
        with open(os.path.join("output/rich_metadata",m), "r") as metaFile:
            data= json.load(metaFile)
            all_dicts.append(data)

    sorted_by_rank = sorted(all_dicts, key=lambda x: x["rarity"]["harmonic"]["rank"])
    rank_edition_dict= {i + 1: data["edition"] for i, data in enumerate(sorted_by_rank)}
    with open(os.path.join("output/","ranks_score.json"), 'w', encoding='utf-8') as outfile:
            json.dump(rank_edition_dict, outfile, indent=2)

    print("dictionary 'rank': 'id' created")





In [86]:
create_jsons()       

Files created in '/layers_data_json' 


In [101]:
validate_schemas_file()

Images sizes check: OK
Weight checks: OK 
No errors in the config schema


In [173]:
#Fill the parameters of the function
generate_collection(num_to_generate=5555,name="Crypto Punks", symbol="PUNKS",description="The punks are coming to get you!")
update_metadatas()

Process completed!


In [ ]:
#Optional!
#Crerate a rich metadata file, calculating means, frequencies, counts and rarity's ranks based on the collection's traits just created
attribute_count=create_counts()
att_freq=calculate_percentages(attribute_count)
create_rich_metadata(attribute_count,att_freq)
mean=calculate_mean()
add_rarity_rank(mean)

In [122]:
summary_rank()

dictionary 'rank': 'id' created
